In [ ]:
import pickle
pickle.HIGHEST_PROTOCOL = 4

import numpy as np
import pandas as pd

import os
from pathlib import Path
import func_proc_filepath as mFILE
from tqdm import tqdm
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def main():
    # root folder
    dir_root = str(Path(Path().resolve()).parent)

    # input: folder path including original images
    dir_au = dir_root + "\\dataset\\expw_images\\au"
    dir_vgg = dir_root + "\\dataset\\expw_images\\vggface"
    
    file_label = dir_root + "\\parameters\\expr_label.csv"
    df_label = pd.read_csv(file_label)
    df_label["file"] = df_label["file"].str.replace(".jpg","")

    # output: folder path
    dir_out = dir_root + "\\dataset\\expw_images\\dataset"
    if os.path.isdir(dir_out) == False:
        os.makedirs(dir_out)

    files_list = mFILE.search_files(dir_vgg, valid_names=[".csv"], invalid_names=["wuert"], ext=None, recursive=False)
    log = "number of files: {0}".format(len(files_list))
    print(log)

    
    df_list = []

    max_cnt = len(files_list)
    #max_cnt = 1000
    for i in tqdm(range(max_cnt)):
        fp = files_list[i]
        name = os.path.splitext(os.path.basename(fp))[0]

        fp_au = dir_au + "\\" + name + ".csv"
        if os.path.exists(fp_au) == False:
            log = "{0}/{1}, {2} is **None**".format(i, max_cnt-1, name)
            print(log)
            continue

        df_au = pd.read_csv(fp_au)
        df_au = df_au.loc[:,df_au.columns.str.contains("frame|AU|pose|gaze_angle")]
        for col in df_au.columns:
            df_au[col]=pd.to_numeric(df_au[col],errors='coerce')
        #df_au = df_au.astype({'a': 'float32', 'c': 'int8'})

        df_vgg = pd.read_csv(fp)
        df_vgg.columns = "vgg-" + df_vgg.columns
        df_vgg = df_vgg.astype("float32")

        df_merge = pd.concat([df_au, df_vgg], axis=1)
        df_merge["file"] = name

        df_list.append(df_merge)

        #log = "{0}/{1}, {2} is finished".format(i, max_cnt-1, name)
        #print(log)

    df_out = pd.concat([x for x in df_list],axis=0)
    df_out.reset_index(drop=True, inplace=True)
    
    df_out = pd.merge(df_label,df_out,on="file") #,how="outer")
    
    file_out = dir_out + "\\expw_with_label.h5"
    df_out.to_hdf(file_out, key='key', mode='w', complevel=5, complib='lzo')

    #df_out = pd.read_hdf(file_out)
    display(df_out)

In [ ]:
if __name__ == "__main__":
    main()